# DD2424 Deep Learning in Data Science
## Assignment 3

### Imports

In [9]:
import sys
!{sys.executable} -m pip install numpy
!{sys.executable} -m pip install matplotlib
!{sys.executable} -m pip install tqdm

In [10]:
import numpy as np
from data import *
from utils import *
from layers.model import Model
from layers.activations import ReLU, Softmax
from layers.dense import Dense
from layers.batchnormalization import BatchNormalization
from layers.dropout import Dropout

### Load data

In [5]:
X_train, Y_train, y = load_data('data_batch_1')
X_mean = np.mean(X_train, axis=1).reshape(X_train.shape[0], 1)
X_std = np.std(X_train, axis=1).reshape(X_train.shape[0], 1)
X_train = normalize(X_train, X_mean, X_std)

assert np.allclose(np.mean(X_train, axis=1).reshape(X_train.shape[0], 1), np.zeros((X_train.shape[0], 1))), \
    "Check normalization, mean should be 0 "
assert np.allclose(np.std(X_train, axis=1).reshape(X_train.shape[0], 1), np.ones((X_train.shape[0], 1))), \
    "Check normalization, std should be 1"

X_val, Y_val, _ = load_data('data_batch_2')
X_test, Y_test, _ = load_data('test_batch')
X_val = normalize(X_val, X_mean, X_std)
X_test = normalize(X_test, X_mean, X_std)

BATCH_SIZE = 20
batch = X_train[:, :BATCH_SIZE], Y_train[:, :BATCH_SIZE]

In [ ]:
X_train.shape

## Exercise 1
Upgrade assignment 2 code to train & test k-layer networks

#### Two-layer network
Checking the gradients

In [ ]:
classifier = Model()
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(ReLU())
classifier.add(Dense(50, 10))
classifier.add(Softmax())

In [ ]:
gradients, _ = classifier.compute_gradients(*batch, 0)
gradients = list(reversed(gradients))

In [ ]:
kernel_gradients_num, bias_gradients_num, _, _ = classifier.compute_gradients_num(*batch, 0)

In [ ]:
equal = True
threshold = 0

for i, (grad_kernel, grad_bias) in enumerate(gradients):
    grad_kernel_num = kernel_gradients_num[i]
    grad_bias_num = bias_gradients_num[i]
    
    equal = equal and np.allclose(grad_kernel[:10, :], grad_kernel_num[:10, :], rtol=1e-6, atol=1e-6)
    equal = equal and np.allclose(grad_bias, grad_bias_num, rtol=1e-6, atol=1e-6)
    
    threshold = max(threshold, np.max(np.abs(grad_kernel[:10, :] - grad_kernel_num[:10, :])))
    threshold = max(threshold, np.max(np.abs(grad_bias - grad_bias_num)))

print(equal)
print(threshold)

#### Three-layer network
Checking the gradients

In [ ]:
classifier = Model()
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(ReLU())
classifier.add(Dense(50, 50))
classifier.add(ReLU())
classifier.add(Dense(50, 10))
classifier.add(Softmax())

In [ ]:
gradients, _ = classifier.compute_gradients(*batch, 0)
gradients = list(reversed(gradients))

In [ ]:
kernel_gradients_num, bias_gradients_num, _, _ = classifier.compute_gradients_num(*batch, 0)

In [ ]:
equal = True
threshold = 0

for i, (grad_kernel, grad_bias) in enumerate(gradients):
    grad_kernel_num = kernel_gradients_num[i]
    grad_bias_num = bias_gradients_num[i]
    
    equal = equal and np.allclose(grad_kernel[:10, :], grad_kernel_num[:10, :], rtol=1e-6, atol=1e-6)
    equal = equal and np.allclose(grad_bias, grad_bias_num, rtol=1e-6, atol=1e-6)
    
    threshold = max(threshold, np.max(np.abs(grad_kernel[:10, :] - grad_kernel_num[:10, :])))
    threshold = max(threshold, np.max(np.abs(grad_bias - grad_bias_num)))

print(equal)
print(threshold)

#### Four-layer network
Checking the gradients

In [ ]:
classifier = Model()
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(ReLU())
classifier.add(Dense(50, 50))
classifier.add(ReLU())
classifier.add(Dense(50, 30))
classifier.add(ReLU())
classifier.add(Dense(30, 10))
classifier.add(Softmax())

In [ ]:
gradients, _ = classifier.compute_gradients(*batch, 0)
gradients = list(reversed(gradients))

In [ ]:
kernel_gradients_num, bias_gradients_num, _, _ = classifier.compute_gradients_num(*batch, 0)

In [ ]:
equal = True
threshold = 0

for i, (grad_kernel, grad_bias) in enumerate(gradients):
    grad_kernel_num = kernel_gradients_num[i]
    grad_bias_num = bias_gradients_num[i]
    
    equal = equal and np.allclose(grad_kernel[:10, :], grad_kernel_num[:10, :], rtol=1e-6, atol=1e-6)
    equal = equal and np.allclose(grad_bias, grad_bias_num, rtol=1e-6, atol=1e-6)
    
    threshold = max(threshold, np.max(np.abs(grad_kernel[:10, :] - grad_kernel_num[:10, :])))
    threshold = max(threshold, np.max(np.abs(grad_bias - grad_bias_num)))

print(equal)
print(threshold)

## Exercise 2
Can I train multi-layer networks?

#### Two-layer network
Replicate assignment 2 results (~46.66%)

In [ ]:
classifier = Model()
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(ReLU())
classifier.add(Dense(50, 10))
classifier.add(Softmax())

In [ ]:
ns = 500
k = 3

loss, accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, k=k, ns=ns, reg_lambda=0.01)

classifier.accuracy(X_test, Y_test)

In [ ]:
steps = 2 * ns * k

train_validation_error(*loss, label='loss', steps=steps)
train_validation_error(*accuracy, label='accuracy', steps=steps)

train_validation_error(*loss, label='loss', steps=steps, save='two-layer-basic')
train_validation_error(*accuracy, label='accuracy', steps=steps, save='two-layer-basic')

#### Three-layer network

In [ ]:
classifier = Model()
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(ReLU())
classifier.add(Dense(50, 50))
classifier.add(ReLU())
classifier.add(Dense(50, 10))
classifier.add(Softmax())

Load all the data for better results

In [11]:
X_train, Y_train, y = load_all_data()
X_mean = np.mean(X_train, axis=1).reshape(X_train.shape[0], 1)
X_std = np.std(X_train, axis=1).reshape(X_train.shape[0], 1)
X_train = normalize(X_train, X_mean, X_std)

assert np.allclose(np.mean(X_train, axis=1).reshape(X_train.shape[0], 1), np.zeros((X_train.shape[0], 1))), \
    "Check normalization, mean should be 0 "
assert np.allclose(np.std(X_train, axis=1).reshape(X_train.shape[0], 1), np.ones((X_train.shape[0], 1))), \
    "Check normalization, std should be 1"

X_val, Y_val, _ = load_all_data(validation=True)
X_test, Y_test, _ = load_data('test_batch')
X_val = normalize(X_val, X_mean, X_std)
X_test = normalize(X_test, X_mean, X_std)

Suggested hyperparameters

In [ ]:
n_batch = 100
eta_min = 1e-5
eta_max = 1e-1
lambda_ = 0.005
k = 2
ns = int(5 * 45_000 / n_batch)

In [ ]:
loss, accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)

classifier.accuracy(X_test, Y_test)

In [ ]:
steps = 2 * ns * k

train_validation_error(*loss, label='loss', steps=steps)
train_validation_error(*accuracy, label='accuracy', steps=steps)

train_validation_error(*loss, label='loss', steps=steps, save='three-layer-basic')
train_validation_error(*accuracy, label='accuracy', steps=steps, save='three-layer-basic')

#### Nine-layer network

In [ ]:
classifier = Model()
# Layer 1
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(ReLU())
# Layer 2
classifier.add(Dense(50, 30))
classifier.add(ReLU())
# Layer 3
classifier.add(Dense(30, 20))
classifier.add(ReLU())
# Layer 4
classifier.add(Dense(20, 20))
classifier.add(ReLU())
# Layer 5
classifier.add(Dense(20, 10))
classifier.add(ReLU())
# Layer 6
classifier.add(Dense(10, 10))
classifier.add(ReLU())
# Layer 7
classifier.add(Dense(10, 10))
classifier.add(ReLU())
# Layer 8
classifier.add(Dense(10, 10))
classifier.add(ReLU())
# Layer 9
classifier.add(Dense(10, 10))
classifier.add(Softmax())

In [ ]:
loss, accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)

classifier.accuracy(X_test, Y_test)

In [ ]:
steps = 2 * ns * k

train_validation_error(*loss, label='loss', steps=steps)
train_validation_error(*accuracy, label='accuracy', steps=steps)

train_validation_error(*loss, label='loss', steps=steps, save='nine-layer-basic')
train_validation_error(*accuracy, label='accuracy', steps=steps, save='nine-layer-basic')

## Exercise 3
Implement batch normalization

#### Three-layer network

In [ ]:
classifier = Model()
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dense(50, 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dense(50, 10))
classifier.add(Softmax())

Checking the gradients

In [ ]:
dense_gradients, batch_gradients = classifier.compute_gradients(*batch, 0)
dense_gradients = list(reversed(dense_gradients))
batch_gradients = list(reversed(batch_gradients))

In [ ]:
kernel_gradients_num, bias_gradients_num, gamma_gradients_num, beta_gradients_num = classifier.compute_gradients_num(*batch, 0)

In [ ]:
equal = True
threshold = 0

for i, (grad_kernel, grad_bias) in enumerate(dense_gradients):
    grad_kernel_num = kernel_gradients_num[i]
    grad_bias_num = bias_gradients_num[i]
    
    equal = equal and np.allclose(grad_kernel[:10, :], grad_kernel_num[:10, :], rtol=1e-6, atol=1e-6)
    equal = equal and np.allclose(grad_bias, grad_bias_num, rtol=1e-6, atol=1e-6)
    
    threshold = max(threshold, np.max(np.abs(grad_kernel[:10, :] - grad_kernel_num[:10, :])))
    threshold = max(threshold, np.max(np.abs(grad_bias - grad_bias_num)))
    
for i, (grad_gamma, grad_beta) in enumerate(batch_gradients):
    grad_gamma_num = gamma_gradients_num[i]
    grad_beta_num = beta_gradients_num[i]
    
    equal = equal and np.allclose(grad_gamma, grad_gamma_num, rtol=1e-6, atol=1e-6)
    equal = equal and np.allclose(grad_beta, grad_beta_num, rtol=1e-6, atol=1e-6)
    
    threshold = max(threshold, np.max(np.abs(grad_gamma - grad_gamma_num)))
    threshold = max(threshold, np.max(np.abs(grad_beta - grad_beta_num)))

print(equal)
print(threshold)

Train the network

In [ ]:
loss, accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)

classifier.accuracy(X_test, Y_test)

In [ ]:
steps = 2 * ns * k
train_validation_error(*loss, label='loss', steps=steps)
train_validation_error(*accuracy, label='accuracy', steps=steps)

##### Tuning lambda
Coarse search

In [ ]:
reg_lambda = [10 ** l for l in np.linspace(-5, -1, 5)]

for i in reg_lambda:
    classifier.initialize_weights()
    classifier.fit(X_train, Y_train, k=k, ns=ns, reg_lambda=i)
    print(f"Accuracy for regularization lambda={i}: {classifier.accuracy(X_test, Y_test)*100}%")

Fine search

In [ ]:
reg_lambda = [10 ** l for l in np.linspace(-3, -1.5, 10)]

for i in reg_lambda:
    classifier.initialize_weights()
    classifier.fit(X_train, Y_train, k=k, ns=ns, reg_lambda=i)
    print(f"Accuracy for regularization lambda={i}: {classifier.accuracy(X_test, Y_test)*100}%")

#### Nine-layer network

In [ ]:
classifier = Model()
# Layer 1
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
# Layer 2
classifier.add(Dense(50, 30))
classifier.add(BatchNormalization(30))
classifier.add(ReLU())
# Layer 3
classifier.add(Dense(30, 20))
classifier.add(BatchNormalization(20))
classifier.add(ReLU())
# Layer 4
classifier.add(Dense(20, 20))
classifier.add(BatchNormalization(20))
classifier.add(ReLU())
# Layer 5
classifier.add(Dense(20, 10))
classifier.add(BatchNormalization(10))
classifier.add(ReLU())
# Layer 6
classifier.add(Dense(10, 10))
classifier.add(BatchNormalization(10))
classifier.add(ReLU())
# Layer 7
classifier.add(Dense(10, 10))
classifier.add(BatchNormalization(10))
classifier.add(ReLU())
# Layer 8
classifier.add(Dense(10, 10))
classifier.add(BatchNormalization(10))
classifier.add(ReLU())
# Layer 9
classifier.add(Dense(10, 10))
classifier.add(Softmax())

In [ ]:
loss, accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)

classifier.accuracy(X_test, Y_test)

In [ ]:
steps = 2 * ns * k
train_validation_error(*loss, label='loss', steps=steps)
train_validation_error(*accuracy, label='accuracy', steps=steps)

## Plots

##### Three-layer network
Include graphs of the evolution of the loss function when you train the 3-layer network with and without batch normalization with the given default parameter setting.

In [ ]:
n_batch = 100
eta_min = 1e-5
eta_max = 1e-1
lambda_ = 0.005
k = 2
ns = int(5 * 45_000 / n_batch)

In [ ]:
classifier = Model()
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(ReLU())
classifier.add(Dense(50, 50))
classifier.add(ReLU())
classifier.add(Dense(50, 10))
classifier.add(Softmax())

In [ ]:
loss, accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)

classifier.accuracy(X_test, Y_test)

In [ ]:
classifier = Model()
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dense(50, 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dense(50, 10))
classifier.add(Softmax())

In [ ]:
bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)

classifier.accuracy(X_test, Y_test)

In [ ]:
steps = 2 * ns * k
compare_batch_plot(*loss, *bn_loss, label='loss', steps=steps)
compare_batch_plot(*accuracy, *bn_accuracy, label='accuracy', steps=steps)

compare_batch_plot(*loss, *bn_loss, label='loss', steps=steps, save='three-layer-comparison')
compare_batch_plot(*accuracy, *bn_accuracy, label='accuracy', steps=steps, save='three-layer-comparison')

##### Nine-layer network
Include graphs of the evolution of the loss function when you train the 3-layer network with and without batch normalization with the given default parameter setting.

In [ ]:
classifier = Model()
# Layer 1
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(ReLU())
# Layer 2
classifier.add(Dense(50, 30))
classifier.add(ReLU())
# Layer 3
classifier.add(Dense(30, 20))
classifier.add(ReLU())
# Layer 4
classifier.add(Dense(20, 20))
classifier.add(ReLU())
# Layer 5
classifier.add(Dense(20, 10))
classifier.add(ReLU())
# Layer 6
classifier.add(Dense(10, 10))
classifier.add(ReLU())
# Layer 7
classifier.add(Dense(10, 10))
classifier.add(ReLU())
# Layer 8
classifier.add(Dense(10, 10))
classifier.add(ReLU())
# Layer 9
classifier.add(Dense(10, 10))
classifier.add(Softmax())

In [ ]:
loss, accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)

classifier.accuracy(X_test, Y_test)

In [71]:
classifier = Model()
# Layer 1
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
# Layer 2
classifier.add(Dense(50, 30))
classifier.add(BatchNormalization(30))
classifier.add(ReLU())
# Layer 3
classifier.add(Dense(30, 20))
classifier.add(BatchNormalization(20))
classifier.add(ReLU())
# Layer 4
classifier.add(Dense(20, 20))
classifier.add(BatchNormalization(20))
classifier.add(ReLU())
# Layer 5
classifier.add(Dense(20, 10))
classifier.add(BatchNormalization(10))
classifier.add(ReLU())
# Layer 6
classifier.add(Dense(10, 10))
classifier.add(BatchNormalization(10))
classifier.add(ReLU())
# Layer 7
classifier.add(Dense(10, 10))
classifier.add(BatchNormalization(10))
classifier.add(ReLU())
# Layer 8
classifier.add(Dense(10, 10))
classifier.add(BatchNormalization(10))
classifier.add(ReLU())
# Layer 9
classifier.add(Dense(10, 10))
classifier.add(Softmax())

In [72]:
bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)

classifier.accuracy(X_test, Y_test)

100%|██████████| 18/18 [03:31<00:00, 11.76s/it]


0.5158

In [73]:
for layer in classifier.layers:
    try:
        print(layer.input_size, layer.output_size)
    except:
        pass

3072 50
50 30
30 20
20 20
20 10
10 10
10 10
10 10
10 10


In [ ]:
steps = 2 * ns * k
compare_batch_plot(*loss, *bn_loss, label='loss', steps=steps)
compare_batch_plot(*accuracy, *bn_accuracy, label='accuracy', steps=steps)

compare_batch_plot(*loss, *bn_loss, label='loss', steps=steps, save='nine-layer-comparison')
compare_batch_plot(*accuracy, *bn_accuracy, label='accuracy', steps=steps, save='nine-layer-comparison')

## Initialization

In [ ]:
sigma_list = [1e-1, 1e-3, 1e-4]

for sigma in sigma_list:
    classifier = Model()
    classifier.add(Dense(X_train.shape[0], 50, initialization="normal", initialization_sigma=sigma))
    classifier.add(ReLU())
    classifier.add(Dense(50, 50, initialization="normal", initialization_sigma=sigma))
    classifier.add(ReLU())
    classifier.add(Dense(50, 10, initialization="normal", initialization_sigma=sigma))
    classifier.add(Softmax())
    
    loss, accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
    print(f"Test accuracy for initialization sigma {sigma}, without batch normalization:"
          f"{classifier.accuracy(X_test, Y_test)}")
    
    classifier = Model()
    classifier.add(Dense(X_train.shape[0], 50, initialization="normal", initialization_sigma=sigma))
    classifier.add(BatchNormalization(50))
    classifier.add(ReLU())
    classifier.add(Dense(50, 50, initialization="normal", initialization_sigma=sigma))
    classifier.add(BatchNormalization(50))
    classifier.add(ReLU())
    classifier.add(Dense(50, 10, initialization="normal", initialization_sigma=sigma))
    classifier.add(Softmax())
    
    bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
    print(f"Test accuracy for initialization sigma {sigma}, batch normalization:"
          f"{classifier.accuracy(X_test, Y_test)}")
    
    steps = 2 * ns * k
    compare_batch_plot(*loss, *bn_loss, label='loss', steps=steps)
    compare_batch_plot(*accuracy, *bn_accuracy, label='accuracy', steps=steps)

    compare_batch_plot(*loss, *bn_loss, label='loss', steps=steps, save=f'sigma-{sigma}'.replace('.', '_'))
    compare_batch_plot(*accuracy, *bn_accuracy, label='accuracy', steps=steps, save=f'sigma-{sigma}'.replace('.', '_'))

## Bonus

In [26]:
n_batch = 100
eta_min = 1e-5
eta_max = 1e-1
lambda_ = 0.005
k = 2
ns = int(5 * 45_000 / n_batch)

### Network architecture
Ideally, we would perform a grid search; due to lack of resources, let's first explore the depth of the network and then the number of nodes.

##### Number of layers
Let's first explore different number of layers with similar node structure.

In [28]:
architectures = [[50, 10],                                # 2-layer
                 [50, 50, 10],                            # 3-layer
                 [50, 40, 20, 10],                        # 4-layer
                 [50, 30, 20, 20, 10],                    # 5-layer
                 [50, 30, 20, 20, 10, 10],                # 6-layer
                 [50, 30, 20, 20, 10, 10, 10],            # 7-layer
                 [50, 30, 30, 20, 10, 10, 10, 10],        # 8-layer
                 [50, 30, 30, 20, 10, 10, 10, 10, 10]]    # 9-layer

for layers in architectures:
    classifier = Model()

    for i in range(len(layers) - 1):
        classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
        classifier.add(BatchNormalization(layers[i]))
        classifier.add(ReLU())

    classifier.add(Dense(layers[-2], layers[-1]))
    classifier.add(Softmax())

    bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)

    print(f"Test accuracy for a {len(layers)}-layer network, in the form of {layers}: "
          f"{classifier.accuracy(X_test, Y_test)}")

  0%|          | 0/18 [00:00<?, ?it/s]

Test accuracy for a 2-layer network, in the form of [50, 10]: 0.5152


  0%|          | 0/18 [00:00<?, ?it/s]

Test accuracy for a 3-layer network, in the form of [50, 50, 10]: 0.5316


  0%|          | 0/18 [00:00<?, ?it/s]

...

Test accuracy for a 7-layer network, in the form of [50, 30, 20, 20, 10, 10, 10]: 0.5215


  0%|          | 0/18 [00:00<?, ?it/s]

Test accuracy for a 8-layer network, in the form of [50, 30, 30, 20, 10, 10, 10, 10]: 0.522


100%|██████████| 18/18 [02:49<00:00,  9.44s/it]


Test accuracy for a 9-layer network, in the form of [50, 30, 30, 20, 10, 10, 10, 10, 10]: 0.5138


##### Number of nodes
Let's now try different node configurations.

In [29]:
architectures = [[100,  50,  30, 10],                        # 314000 weights, 190 nodes -- 55.23% accuracy
                 [ 50,  40,  20, 10],                        # 156600 weights, 120 nodes -- 53.77% accuracy
                 [ 50,  50,  50, 10],                        # 159100 weights, 160 nodes -- 53.78% accuracy
                 [100, 100,  10, 10],                        # 318300 weights, 220 nodes -- 55.06% accuracy
                 [ 30,  50, 100, 10],                        #  99660 weights, 190 nodes -- 52.31% accuracy
                 [ 10, 100,  50, 10],                        #  37220 weights, 170 nodes -- 47.80% accuracy
                 [ 50, 150,  50, 10]]                        # 169100 weights, 260 nodes -- 55.00% accuracy


for layers in architectures:
    classifier = Model()

    for i in range(len(layers) - 1):
        classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
        classifier.add(BatchNormalization(layers[i]))
        classifier.add(ReLU())

    classifier.add(Dense(layers[-2], layers[-1]))
    classifier.add(Softmax())

    bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)

    print(f"Test accuracy for a {len(layers)}-layer network, in the form of {layers}: "
          f"{classifier.accuracy(X_test, Y_test)}")

  0%|          | 0/18 [00:00<?, ?it/s]

Test accuracy for a 4-layer network, in the form of [100, 50, 30, 10]: 0.5523


  0%|          | 0/18 [00:00<?, ?it/s]

Test accuracy for a 4-layer network, in the form of [50, 50, 50, 10]: 0.5378


  0%|          | 0/18 [00:00<?, ?it/s]

...

Test accuracy for a 4-layer network, in the form of [30, 50, 100, 10]: 0.5231


  0%|          | 0/18 [00:00<?, ?it/s]

Test accuracy for a 4-layer network, in the form of [10, 100, 50, 10]: 0.478


100%|██████████| 18/18 [02:56<00:00,  9.79s/it]


Test accuracy for a 4-layer network, in the form of [50, 150, 50, 10]: 0.55


### Apply activation before BN

##### Three-layer network

In [40]:
classifier = Model()
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dense(50, 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dense(50, 10))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

100%|██████████| 18/18 [02:08<00:00,  7.11s/it]


0.5316

In [41]:
classifier = Model()
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(ReLU())
classifier.add(BatchNormalization(50))
classifier.add(Dense(50, 50))
classifier.add(ReLU())
classifier.add(BatchNormalization(50))
classifier.add(Dense(50, 10))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

100%|██████████| 18/18 [02:51<00:00,  9.53s/it]


0.5296

##### Nine-layer network

In [43]:
layers = [50, 30, 20, 20, 10, 10, 10, 10, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(BatchNormalization(layers[i]))
    classifier.add(ReLU())
    
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

100%|██████████| 18/18 [02:40<00:00,  8.90s/it]


0.5158

In [44]:
layers = [50, 30, 20, 20, 10, 10, 10, 10, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(ReLU())
    classifier.add(BatchNormalization(layers[i]))
    
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

100%|██████████| 18/18 [03:04<00:00, 10.24s/it]


0.5199

##### Best network

In [45]:
layers = [100,  50,  30, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(BatchNormalization(layers[i]))
    classifier.add(ReLU())
    
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

100%|██████████| 18/18 [04:38<00:00, 15.49s/it]


0.5523

In [46]:
layers = [100,  50,  30, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(ReLU())
    classifier.add(BatchNormalization(layers[i]))
    
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

100%|██████████| 18/18 [03:55<00:00, 13.06s/it]


0.5475

### Dropout

##### Three-layer network

In [47]:
classifier = Model()
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dense(50, 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dense(50, 10))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

100%|██████████| 18/18 [02:01<00:00,  6.73s/it]


0.5316

In [48]:
classifier = Model()
classifier.add(Dropout(0.5))
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dropout(0.5))
classifier.add(Dense(50, 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dropout(0.5))
classifier.add(Dense(50, 10))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

100%|██████████| 18/18 [03:53<00:00, 12.97s/it]


0.4375

In [ ]:
classifier = Model()
classifier.add(Dropout(0.5))
classifier.add(Dense(X_train.shape[0], 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dropout(0.5))
classifier.add(Dense(50, 50))
classifier.add(BatchNormalization(50))
classifier.add(ReLU())
classifier.add(Dropout(0.5))
classifier.add(Dense(50, 10))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=3*k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

 47%|████▋     | 26/55 [07:42<06:49, 14.13s/it]

##### Nine-layer network

In [ ]:
layers = [50, 30, 20, 20, 10, 10, 10, 10, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(BatchNormalization(layers[i]))
    classifier.add(ReLU())
    
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

In [ ]:
layers = [50, 30, 20, 20, 10, 10, 10, 10, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dropout(0.5))
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(BatchNormalization(layers[i]))
    classifier.add(ReLU())
    
classifier.add(Dropout(0.5))
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

In [ ]:
layers = [50, 30, 20, 20, 10, 10, 10, 10, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dropout(0.5))
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(BatchNormalization(layers[i]))
    classifier.add(ReLU())
    
classifier.add(Dropout(0.5))
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=3*k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

##### Best network

In [ ]:
layers = [100,  50,  30, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(BatchNormalization(layers[i]))
    classifier.add(ReLU())
    
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

In [ ]:
layers = [100,  50,  30, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dropout(0.5))
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(BatchNormalization(layers[i]))
    classifier.add(ReLU())
    
classifier.add(Dropout(0.5))
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

In [ ]:
layers = [100,  50,  30, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dropout(0.5))
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(BatchNormalization(layers[i]))
    classifier.add(ReLU())
    
classifier.add(Dropout(0.5))
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=3*k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

##### Wider network

In [ ]:
layers = [200, 100,  60, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dropout(0.5))
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(BatchNormalization(layers[i]))
    classifier.add(ReLU())
    
classifier.add(Dropout(0.5))
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

In [ ]:
layers = [200, 100,  60, 10]

classifier = Model()

for i in range(len(layers) - 1):
    classifier.add(Dropout(0.5))
    classifier.add(Dense(layers[i - 1] if i > 0 else X_train.shape[0], layers[i]))
    classifier.add(BatchNormalization(layers[i]))
    classifier.add(ReLU())
    
classifier.add(Dropout(0.5))
classifier.add(Dense(layers[-2], layers[-1]))
classifier.add(Softmax())

bn_loss, bn_accuracy = classifier.fit(X_train, Y_train, X_val, Y_val, ns=ns, k=3*k, reg_lambda=lambda_)
classifier.accuracy(X_test, Y_test)

### Data augmentation